In [9]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd
from scipy.io import arff
data = arff.loadarff('slump.arff')
df = pd.DataFrame(data[0])

covariables = df.iloc[:,0:7].values
response = df.iloc[:,7:10].values
positions = np.arange(103)

from sklearn.model_selection import train_test_split

covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


##APPRENTISSAGE

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = numpy.empty(a.shape, dtype=a.dtype)
    shuffled_b = numpy.empty(b.shape, dtype=b.dtype)
    permutation = numpy.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

import tensorflow as tf 




# Import data
# Tensorflow is finicky about shapes, so resize


X_datatrain, X_datatest, targets_train, targets_test = covariables_train, covariables_test, response_train, response_test

#les dimensions de X for training
NbrLignes_train = X_datatrain.shape[0]  ###
NbrColonnes_train= X_datatrain.shape[1] ###

#les dimensions de X for testing
NbrLignes_test = X_datatest.shape[0]  ###
NbrColonnes_test = X_datatest.shape[1] ###
#Y_data = targets 

Y_datatrain = tf.reshape(targets_train, [NbrLignes_train,3])
Y_datatest = tf.reshape(targets_test, [NbrLignes_test,3])

#build the model

# input X: batch_size x NbrColonnes, the first dimension (None) will index the data in the mini-batch
Xfill = tf.placeholder(tf.float32, shape= [None, NbrColonnes_train])
# correct answers will go here
Yfill = tf.placeholder(tf.float32, shape=[None, 3]) 
#Let's add some layers

# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 160
N = 100
O = 30
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([NbrColonnes_train, L], stddev=0.1))  # 
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
B3 = tf.Variable(tf.ones([N])/10)
W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.ones([O])/10)
W5 = tf.Variable(tf.truncated_normal([O, 3], stddev=0.1))
B5 = tf.Variable(tf.zeros([3]))

# The model, with dropout at each layer


Y1 = tf.nn.relu(tf.matmul(Xfill, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
Y2d = tf.nn.dropout(Y2, pkeep)

Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
Y3d = tf.nn.dropout(Y3, pkeep)

Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
Y4d = tf.nn.dropout(Y4, pkeep)

Y_ = tf.matmul(Y4d, W5) + B5

cross_entropy = tf.losses.mean_squared_error(Yfill, Y_)

# accuracy of the trained model, between 0 (worst) and 1 (best)
#Modification de l'accuracy 

#calcul de l'accuracy, une nouvelle manière: :  

# variable learning rate
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy) 

# init
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 100.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

train_loss = []
test_loss = []

# Gradient descent loop for 500 steps
for i in range(5000):
 # Select random minibatch
    j = 1
    ##### shuffling our data::: really important
    X_datasf, Y_datasf = shuffle_in_unison(X_datatrain, Y_datatrain.eval())
    
    ##### defining a decreasing learning rate: 
    # learning rate decay
    learning_rate = 0.0001001329829992624
    #learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    ##### 
    for start, end in zip(range(0, NbrLignes_train, 52), range(20, NbrLignes_train, 52)):

             X_batch,Y_batch = X_datasf[start:end,],Y_datasf[start:end,]
             j +=1
             # Do gradient descent step
             _, loss_val_train = sess.run([train_step, cross_entropy], feed_dict={Xfill: X_batch, Yfill: Y_batch, pkeep: 0.75, lr: learning_rate})
             if i %20==0:    
                 
             
                #if j%20==0:
                 loss_val_test = sess.run(cross_entropy, feed_dict={Xfill: X_datatest, Yfill: Y_datatest.eval(), pkeep: 1.0, lr:  learning_rate})
                 train_loss.append(loss_val_train)
                 
                 test_loss.append(loss_val_test)
                 print('LOSS at step %s: - Train loss value : %f -- Test loss value : %f' % (i, loss_val_train,loss_val_test))
#print(test_loss)
#print(test_loss)
#LOSS at step 420: - Train loss value : 22.385496 -- Test loss value : 9.721767

/usr/local/misc/Python/Python-3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/usr/local/misc/Python/Python-3.6.7/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


LOSS at step 0: - Train loss value : 2648.545166 -- Test loss value : 1968.223511
LOSS at step 0: - Train loss value : 2907.702637 -- Test loss value : 1865.133667
LOSS at step 20: - Train loss value : 1097.554321 -- Test loss value : 522.088867
LOSS at step 20: - Train loss value : 1040.938843 -- Test loss value : 513.570190
LOSS at step 40: - Train loss value : 715.231384 -- Test loss value : 423.931946
LOSS at step 40: - Train loss value : 621.348816 -- Test loss value : 424.582611
LOSS at step 60: - Train loss value : 761.149048 -- Test loss value : 426.759216
LOSS at step 60: - Train loss value : 778.664307 -- Test loss value : 428.861450
LOSS at step 80: - Train loss value : 690.229187 -- Test loss value : 373.820923
LOSS at step 80: - Train loss value : 603.710815 -- Test loss value : 371.508240
LOSS at step 100: - Train loss value : 524.154114 -- Test loss value : 350.718964
LOSS at step 100: - Train loss value : 546.477905 -- Test loss value : 350.615082
LOSS at step 120: - Tr

LOSS at step 1080: - Train loss value : 255.095322 -- Test loss value : 379.104034
LOSS at step 1080: - Train loss value : 244.914871 -- Test loss value : 378.472260
LOSS at step 1100: - Train loss value : 186.303909 -- Test loss value : 366.129272
LOSS at step 1100: - Train loss value : 230.126419 -- Test loss value : 366.130859
LOSS at step 1120: - Train loss value : 263.565613 -- Test loss value : 405.819458
LOSS at step 1120: - Train loss value : 179.826141 -- Test loss value : 407.617920
LOSS at step 1140: - Train loss value : 276.055176 -- Test loss value : 377.442139
LOSS at step 1140: - Train loss value : 174.595535 -- Test loss value : 373.502777
LOSS at step 1160: - Train loss value : 246.531769 -- Test loss value : 371.306305
LOSS at step 1160: - Train loss value : 213.242844 -- Test loss value : 368.135345
LOSS at step 1180: - Train loss value : 181.663483 -- Test loss value : 352.177826
LOSS at step 1180: - Train loss value : 206.509766 -- Test loss value : 354.674500
LOSS

LOSS at step 2160: - Train loss value : 237.511261 -- Test loss value : 336.975250
LOSS at step 2160: - Train loss value : 219.205093 -- Test loss value : 339.701019
LOSS at step 2180: - Train loss value : 144.460419 -- Test loss value : 325.639526
LOSS at step 2180: - Train loss value : 164.438019 -- Test loss value : 326.488159
LOSS at step 2200: - Train loss value : 173.937042 -- Test loss value : 314.235626
LOSS at step 2200: - Train loss value : 207.428360 -- Test loss value : 314.705627
LOSS at step 2220: - Train loss value : 227.205231 -- Test loss value : 331.454132
LOSS at step 2220: - Train loss value : 239.076004 -- Test loss value : 334.844543
LOSS at step 2240: - Train loss value : 151.558899 -- Test loss value : 338.336548
LOSS at step 2240: - Train loss value : 267.319672 -- Test loss value : 339.887909
LOSS at step 2260: - Train loss value : 202.506638 -- Test loss value : 337.653259
LOSS at step 2260: - Train loss value : 125.393753 -- Test loss value : 339.260681
LOSS

LOSS at step 3240: - Train loss value : 157.544571 -- Test loss value : 289.376343
LOSS at step 3240: - Train loss value : 201.407745 -- Test loss value : 290.674194
LOSS at step 3260: - Train loss value : 126.109375 -- Test loss value : 267.680420
LOSS at step 3260: - Train loss value : 195.024109 -- Test loss value : 268.484985
LOSS at step 3280: - Train loss value : 156.598724 -- Test loss value : 278.105835
LOSS at step 3280: - Train loss value : 161.956497 -- Test loss value : 275.824982
LOSS at step 3300: - Train loss value : 147.738739 -- Test loss value : 303.748810
LOSS at step 3300: - Train loss value : 171.856522 -- Test loss value : 304.495209
LOSS at step 3320: - Train loss value : 175.507614 -- Test loss value : 288.736206
LOSS at step 3320: - Train loss value : 146.347702 -- Test loss value : 290.512726
LOSS at step 3340: - Train loss value : 121.478630 -- Test loss value : 297.834320
LOSS at step 3340: - Train loss value : 107.974953 -- Test loss value : 298.070892
LOSS

LOSS at step 4320: - Train loss value : 137.254898 -- Test loss value : 272.287201
LOSS at step 4320: - Train loss value : 198.909271 -- Test loss value : 270.776886
LOSS at step 4340: - Train loss value : 142.749542 -- Test loss value : 285.122742
LOSS at step 4340: - Train loss value : 133.385025 -- Test loss value : 282.827057
LOSS at step 4360: - Train loss value : 95.156624 -- Test loss value : 247.081345
LOSS at step 4360: - Train loss value : 145.755371 -- Test loss value : 245.603226
LOSS at step 4380: - Train loss value : 247.336258 -- Test loss value : 255.613754
LOSS at step 4380: - Train loss value : 87.587860 -- Test loss value : 249.159714
LOSS at step 4400: - Train loss value : 184.719925 -- Test loss value : 266.529358
LOSS at step 4400: - Train loss value : 142.056152 -- Test loss value : 270.095337
LOSS at step 4420: - Train loss value : 190.692322 -- Test loss value : 263.735443
LOSS at step 4420: - Train loss value : 140.530701 -- Test loss value : 261.543213
LOSS a

# Let us compute the aRRMSE

In [10]:
learning_rate = 0.0001001329829992624

Y_testpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatest, tf.float32).eval(), Yfill: tf.cast(Y_datatest, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})
#numpy.savetxt('Y_testpred_with_stock_%s_and_ai_%s_modelcut_%s_vartimeframe_%s.txt'%(stock, ai, cut, vartimeframe), Y_testpred, fmt='%f')
Y_trainpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatrain, tf.float32).eval(), Yfill: tf.cast(Y_datatrain, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})


In [11]:
Y_testpred

array([[12.296146 , 34.893208 , 30.414185 ],
       [11.781056 , 32.261723 , 24.179066 ],
       [13.292901 , 34.60828  , 17.925142 ],
       [10.104949 , 29.140244 , 26.763617 ],
       [11.720647 , 31.629486 , 20.722048 ],
       [12.233914 , 33.004753 , 21.418697 ],
       [ 4.7921243, 15.741513 , 25.799324 ],
       [13.029567 , 34.03006  , 18.372917 ],
       [16.654474 , 43.550407 , 21.774126 ],
       [14.964795 , 39.756718 , 24.234705 ],
       [ 8.181367 , 24.92323  , 27.531197 ],
       [19.602608 , 52.52906  , 33.022785 ],
       [20.79553  , 55.292828 , 33.03003  ],
       [10.144857 , 27.216574 , 19.2749   ],
       [10.987417 , 29.162989 , 18.494675 ],
       [15.787713 , 41.20526  , 20.786592 ],
       [13.110942 , 34.30029  , 19.419786 ],
       [13.622944 , 36.826923 , 25.099596 ],
       [17.283867 , 45.213917 , 22.724628 ],
       [15.626089 , 42.194233 , 28.253662 ],
       [ 6.7807565, 21.777218 , 34.36345  ]], dtype=float32)

In [18]:
from numpy import linalg as LA

In [19]:

aRRMSE_list = []

for i in [0,1,2] :
    
    Errors_test =  Y_testpred[:,i] - response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    ytrain_mean_i = np.tile(Y_trainpred[:,i].mean(axis = 0), (covariables_test.shape[0], 1))

    Errors_relative = ytrain_mean_i - response_test[:,i]

    Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%Final_SPNNR_aRRMSE)

    aRRMSE_list.append(Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.957201
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.145412
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.770048


In [20]:
np.mean(aRRMSE_list)

18.624220293491167

# Check if old way calculation is wrong

In [22]:
Errors_test =  Y_testpred - response_test


print('<*********************************Let us check the aRRMSE*********************************>')

ytrain_mean = np.tile(Y_trainpred.mean(axis = 0), (response_test.shape[0], 1))

Errors_relative = ytrain_mean - response_test

Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

print("Final_Partition aRRMSE is : %f"%Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final_Partition aRRMSE is : 84.178972


*** The answer is yes ***

# Checking True aRRMSE of S-SPNNR on Slump

In [6]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd
from numpy import linalg as LA
from sklearn.model_selection import train_test_split

from scipy.io import arff
data = arff.loadarff('slump.arff')
df = pd.DataFrame(data[0])

covariables = df.iloc[:,0:7].values
response = df.iloc[:,7:10].values
positions = np.arange(103)


covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


In [7]:
spnnr_covariables_train, spnnr_covariables_test, spnnr_response_train, spnnr_response_test,spnnr_positions_train,spnnr_positions_test = train_test_split(covariables, response,positions, test_size=0.33, random_state=42)

In [8]:
spnnr_Y_testpred =np.loadtxt('All_Y_test_predictions.out', delimiter=',' )
spnnr_Y_trainpred =np.loadtxt('All_Y_train_predictions.out', delimiter=',' )

In [9]:
spnnr_aRRMSE_list = []

for i in [0,1,2] :
    
    spnnr_Errors_test =  spnnr_Y_testpred[:,i] - spnnr_response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    spnnr_ytrain_mean_i = np.tile(spnnr_Y_trainpred[:,i].mean(axis = 0), (spnnr_covariables_test.shape[0], 1))

    spnnr_Errors_relative = spnnr_ytrain_mean_i - spnnr_response_test[:,i]

    spnnr_Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(spnnr_Errors_test)**2)/(LA.norm(spnnr_Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%spnnr_Final_SPNNR_aRRMSE)

    spnnr_aRRMSE_list.append(spnnr_Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 12.755554
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 12.263640
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 12.938749


In [10]:
np.mean(spnnr_aRRMSE_list)

12.652647644355206